# **AI-Powered Talent Sourcing and Ranking System**

🎯 **Executive Summary** :
In the dynamic landscape of talent acquisition, manually identifying and ranking suitable candidates is a labor-intensive and time-consuming process, prone to human biases and inefficiencies. This project outlines a cutting-edge Machine Learning (ML) powered pipeline designed to revolutionize the candidate sourcing and management efforts for technology companies. By leveraging Large Language Models (LLMs) and sophisticated ranking algorithms, we aim to automate the initial fitness assessment, provide dynamic re-ranking capabilities based on human feedback, and offer strategies to mitigate bias, ultimately saving significant time and enabling the spotting of high-potential candidates more effectively.

Our current talent sourcing and management operations face several critical challenges:

1. **Deep Role Understanding is Key:** 🔑 Accurately matching candidates requires a profound understanding of the client's needs and the nuances of a specific role, which is often difficult to capture comprehensively.

2. **Identifying "Shine" Factors:** 🌟 Discerning what truly makes a candidate exceptional for a role goes beyond just keywords and demands nuanced evaluation.

3. **Talent Discovery:** 🗺️ Identifying where to locate truly talented individuals within vast professional networks is a persistent hurdle.

4. **Manual & Labor-Intensive Processes:** ⏳ The core of our job relies heavily on manual operations, leading to high human labor costs and potential for bottlenecks.

5. **Subjectivity & Bias:**  Manual review processes, while valuable, can introduce human biases, potentially overlooking diverse and highly qualified candidates.

6. **Static Ranking:** 📉 Our current ranking approach lacks adaptability; when a preferred candidate isn't at the top of the initial list, manually re-evaluating the entire list is inefficient.

We seek a robust, automated solution to streamline these challenges, allowing our expert recruiters to focus on high-value interactions rather than tedious initial screening.









#### **📥 Data Ingestion & Preprocessing :Mount Google Drive and Load CSV**

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd

file_path = '/content/drive/MyDrive/Colab Notebooks/potential-talents.csv'
df = pd.read_csv(file_path)
df.head()

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


#### **Clean Data (Remove Duplicates)**

In [9]:
df =  df.drop_duplicates(subset=["location", "job_title", "connection"], keep="first")
df.shape

(53, 5)

#### **Create Candidate Profile Strings**

In [10]:
#  Create simple text profiles for each candidate
def create_profile(row):
    """Combine candidate info into an easy-to-read format"""
    return f"""
    Candidate Details:
    - Job: {row['job_title']}
    - Works in: {row['location']}
    - Has {row['connection']} connections
    """

df["candidate_profile"] = df.apply(create_profile, axis=1)
#print(df["candidate_profile"].iloc[0])
df.head(3)

,id,job_title,location,connection,fit,candidate_profile
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN,\n Candidate Details:\n - Job: 2019 C.T....
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN,\n Candidate Details:\n - Job: Native En...
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,\n Candidate Details:\n - Job: Aspiring ...


This is where the actual "candidate profile" strings are generated and added as a new column to your DataFrame. This new column is a crucial input for any subsequent LLM processing, as it provides a unified text representation of each candidate.



## **🧠 Initial Candidate Ranking (LLM-Powered)**

### **Method 1: Using GEMMA 3 + LoRA fine tuning model via Unsloth**

#### Install Required Packages **(Unsloth + Dependencies)**

To enable efficient fine-tuning and inference of **large language models using LoRA (Low-Rank Adaptation)**, we use the **Unsloth library**. This setup allows for memory-efficient 4-bit quantized model loading and supports state-of-the-art models like Mistral, Gemma, and Qwen.

The following installation steps configure the environment with all necessary components:
1. **unsloth:** Main library to patch and optimize LLMs for fast fine-tuning and inference.
2. **unsloth_zoo:** Contains model-specific enhancements and patch utilities.
3. **bitsandbytes:** Enables 4-bit quantization for reduced memory usage.
4. **peft, trl:** Tools for efficient parameter-efficient fine-tuning (PEFT), including LoRA.
5. **xformers, triton, cut_cross_entropy:** Provide low-level GPU-optimized operations.
6. **sentencepiece, protobuf:** Required for tokenization and model compatibility.
7. **datasets, huggingface_hub, hf_transfer:** Support dataset loading and model hosting.


In [ ]:
# Install Unsloth & Dependencies

!pip install --no-deps unsloth
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" huggingface_hub hf_transfer
!pip install --upgrade pip
!pip install unsloth
!pip install sentencepiece protobuf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.5/297.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver 

#### **Load Gemma 3 4B with Unsloth + LoRA**

🧠 **Model Loading & LoRA Configuration**
To enable efficient candidate evaluation using a large language model (LLM), we utilize Unsloth's implementation of the Gemma 3 4B IT model with 4-bit quantization and LoRA (Low-Rank Adaptation) fine-tuning.

This section of the code performs two key steps:

1. **Model Initialization**
We load the unsloth/gemma-3-4b-it-bnb-4bit model using the FastLanguageModel.from_pretrained() function. This method:

Loads the model in **4-bit precision**, drastically reducing memory consumption.

Sets a **2048** token context window, suitable for most candidate profile evaluations.

Ensures compatibility with **PEFT** and fast inference by enabling Unsloth's internal optimizations.

2. **LoRA Application**
We apply LoRA on key transformer components (q_proj, k_proj, v_proj, o_proj, etc.) using FastLanguageModel.get_peft_model(). This method:

Injects trainable adapter weights to support fine-tuning without altering the base model.

**Uses:**

**r = 32**: LoRA rank, defining the low-rank approximation size.

**lora_alpha = 32**: Scaling factor for LoRA layers.

**lora_dropout** = 0.0: No dropout applied during fine-tuning.

**use_gradient_checkpointing = "unsloth"**: Enables memory-efficient training for long sequences.

This configuration allows the model to be adapted efficiently to downstream tasks (like candidate scoring) while maintaining minimal compute cost—ideal for practical deployment in resource-constrained environments like Colab or small GPUs.

In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it-bnb-4bit",  # Or any other 4B LoRA model
    max_seq_length = 2048,
    load_in_4bit = True,
    full_finetuning = False,
)


# ✅ Apply LoRA (Lightweight Fine-tuning)
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,
    lora_dropout = 0.0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    use_rslora = False,
    loftq_config = None,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.7.3: Fast Gemma3 patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/3.23G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

Unsloth: Making `base_model.model.model.vision_tower.vision_model` require gradients


#### **Define Candidate Scoring Function**

To evaluate how well a candidate matches a target job role, we define a scoring function using our fine-tuned LLM (**Gemma 3 4B via Unsloth + LoRA**).

🔧**score_candidate(candidate_profile: str, target_role: str)** -> str
This function performs structured candidate evaluation by prompting the model with a specific query that emulates how a human recruiter would assess a candidate. Here's how it works:

In [ ]:
def score_candidate(candidate_profile: str, target_role: str) -> str:
    prompt = f"""
You are an expert HR recruiter.

Evaluate the following candidate for the role of "{target_role}".
Provide a score from 1 to 10 only (just the number, no explanation).

Candidate Profile:
{candidate_profile}

Score:"""

    # Correct message structure
    messages = [
        {"role": "user", "content": [{"type": "text", "text": prompt.strip()}]}
    ]

    # ✅ Return dict so `**inputs` works
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt",
        return_dict=True   # ← important!
    )

    # Move tensors to correct device
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=10,
        do_sample=False
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    last_line = decoded.strip().split("\n")[-1]
    score = ''.join(filter(str.isdigit, last_line))

    return score if score else "0"


**🎯 Candidate Scoring Mechanism**
This process leverages the fine-tuned LLM to automatically assign a fitness score to each candidate for a given role.

**Prompt Engineering:**📝 A precise prompt instructs the LLM, acting as an "expert HR recruiter," to evaluate a candidate for a specific target_role (e.g., "Aspiring Hiring Manager") and generate only a numerical score from 1 to 10, without explanation.

**Message Formatting & Tokenization:** 💬 The prompt is encased in a structured message format compatible with the LLM's chat template. This ensures proper instruction adherence. These messages are then converted into numerical tokens and transferred to the GPU for efficient processing.

**LLM Inference:** 🚀 The model generates a response using deterministic decoding (do_sample=False) and a limited number of new tokens (max_new_tokens=10), as only a score is expected.

**Score Extraction:** ✅ The generated text is parsed to extract the numerical score. A default of "0" is assigned if no valid number is found.

**Output:** A stringified score from "0" to "10," representing the candidate's fitness for the target role.

**Purpose:** This automation is crucial for efficient candidate ranking, enabling rapid sorting, shortlisting for human review, and significantly boosting recruiter productivity through AI augmentation.

#### **Load Candidates Data and Generate Scores**

🚀 Batch Scoring with Progress Monitoring
To evaluate all candidate profiles for the role of "Human Resources Manager", we implement a robust and efficient batch scoring pipeline.

In [ ]:
target_role = "Human Resources Manager"
from tqdm import tqdm
tqdm.pandas()

def safe_score(profile):
    try:
        return score_candidate(profile, target_role)
    except Exception as e:
        return None

df["gemma_score"] = df["candidate_profile"].progress_apply(safe_score)



100%|██████████| 53/53 [00:38<00:00,  1.37it/s]


📊 Sample Output: AI-Generated Candidate Scores
The table below illustrates the first 10 entries from the candidate dataset along with their corresponding gemma_score, which represents the AI-generated relevance rating for the role of Human Resources Manager.

🧠  Output Represents:

**job_title:** The professional title or self-described role from the candidate's profile.

**gemma_score:** A score from 1 to 10 generated by the fine-tuned Gemma 3 4B model, representing how well the candidate aligns with the target role of Human Resources Manager.

In [ ]:
df[["job_title", "gemma_score"]].head(10)

,job_title,gemma_score
0,2019 C.T. Bauer College of Business Graduate (...,7
1,Native English Teacher at EPIK (English Progra...,6
2,Aspiring Human Resources Professional,3
3,People Development Coordinator at Ryan,7
4,Advisory Board Member at Celal Bayar University,3
5,Aspiring Human Resources Specialist,3
6,Student at Humber College and Aspiring Human R...,3
7,HR Senior Specialist,7
9,Seeking Human Resources HRIS and Generalist Po...,7
10,Student at Chapman University,3



**Higher scores (e.g., 7):** Indicate candidates with more relevant titles such as "HR Senior Specialist" or "People Development Coordinator", which suggest direct HR experience.

**Moderate scores (e.g., 3):** Often correspond to students or individuals using aspirational language (e.g., "Aspiring Human Resources Professional"), indicating potential but limited current experience.

Scores reflect semantic understanding, not just keyword matching, making this approach suitable for nuanced screening.

## **Method 2: Candidate ranking using GROK with LoRA via Unsloth**

####  Install and Import Unsloth with GROK Model

To enable efficient model loading, quantization, and fine-tuning using the Unsloth framework, we installed the core and supporting libraries:

**unsloth:** Enables fast and memory-efficient LoRA fine-tuning.

**transformers, peft, trl:** Core libraries for working with LLMs.

**bitsandbytes, xformers:** Support 4-bit quantization and optimized attention.

**accelerate, einops:** Help with training and tensor operations.

The **--no-deps** flag avoids version conflicts in Colab.

In [5]:
!pip install --no-deps unsloth
!pip install einops xformers accelerate bitsandbytes peft trl transformers


#### Load GROK-1 with Unsloth

To efficiently load and prepare the **Mistral-7B-Instruct v0.3** model with **4-bit quantization** using Unsloth:

- **unsloth/mistral-7b-instruct-v0.3-bnb-4bit:** A 4-bit quantized variant
optimized for fast inference and fine-tuning.

- **load_in_4bit=True:** Minimizes memory usage, making large models more accessible on limited hardware.

- **max_seq_length=2048:** Sets the maximum token input length the model can handle per sample.

This step initializes both the model and its tokenizer for downstream tasks such as candidate ranking or conversational AI.

In [7]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = True,
)


==((====))==  Unsloth 2025.7.3: Fast Mistral patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


#### Apply LoRA for fine-tuning

To enable efficient parameter-efficient fine-tuning **(PEFT)** of the **Mistral model** using **LoRA (Low-Rank Adaptation)**:

Applies LoRA to selected attention layers (query/key/value/output projections).
- **r=16:** Sets the rank for low-rank updates, balancing memory usage and learning capacity.
- **lora_alpha=16:** Controls the scaling of LoRA weights.
- **lora_dropout=0:** No dropout applied (tuned for performance).
- **use_gradient_checkpointing="unsloth"**: Enables memory-efficient training using Unsloth's optimization.

This setup reduces the number of trainable parameters while retaining fine-tuning effectiveness on downstream tasks (.g., candidate ranking).

In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=16,
    lora_dropout=0,
    use_gradient_checkpointing="unsloth",
    random_state=42,
)


Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.7.3 patched 32 layers with 32 QKV layers, 32 O layers and 0 MLP layers.


#### Define Scoring Prompt Function

The score_candidate(profile, role) function is responsible for evaluating a candidate's suitability for a given job role using a fine-tuned LLM.
- A structured prompt is dynamically generated with the candidate’s profile and target role.
- The prompt is tokenized and fed into the model using a chat template.
- The model performs a deterministic generation (do_sample=False) to ensure consistent scoring.
- The final score is extracted from the last line of the model’s decoded response using basic string filtering.

In [34]:
def score_candidate(profile, role):
    prompt = f"""
You are an expert HR recruiter.

Evaluate the following candidate for the specific role of "{role}".
Give a score from 1 to 10 ONLY (just the number) based on **how well the candidate's job title, experience, and profile match the role of '{role}'**.

Candidate Profile:
{profile}

Score:"""

    messages = [
        {"role": "user", "content": prompt.strip()}
    ]

    # Get tokenized input
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True,
        tokenize=True
    )

    # Move to correct device
    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)

    # Generate
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=10,
        do_sample=False
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    last_line = decoded.strip().split("\n")[-1]
    score = ''.join(filter(str.isdigit, last_line))

    return score if score else "0"


#### Apply to the DataFrame

In [35]:
df["grok_score"] = df["candidate_profile"].apply(lambda x: score_candidate(x, "Aspiring Hiring Manager"))


✅ **Top 20** Ranked Candidates by **GROK** Score
This output represents the top 20 candidate profiles as ranked by a GROK-based LLM fine-tuned with LoRA for the role of Human Resources Manager.

In [36]:
top_20 = df.sort_values(by="grok_score", ascending=False).head(20)
print(top_20[["job_title", "grok_score"]])

                                             job_title grok_score
88                     Director Human Resources  at EY          9
68   Director of Human Resources North America, Gro...          9
77              Human Resources Generalist at Schwan's          8
66   Human Resources, Staffing and Recruiting Profe...          8
102                     Always set them up for Success          8
93   Seeking Human  Resources Opportunities. Open t...          8
90        Lead Official at Western Illinois University          8
83   Human Resources professional for the world lea...          8
82           HR Manager at Endemol Shine North America          8
80   Senior Human Resources Business Partner at Hei...          8
76   Human Resources|\nConflict Management|\nPolici...          8
70     Human Resources Generalist at ScottMadden, Inc.          8
69   Retired Army National Guard Recruiter, office ...          8
67             Human Resources Specialist at Luxottica          8
103   Dire

**Interpretation:**
- All top candidates scored 8 or 9, indicating a high relevance to the HR Manager role.
- Common themes among the top-ranked titles include:
1. Director of Human Resources
2. HR Manager
3. Human Resources Business Partner
4. People Development Coordinator

These titles align closely with leadership or operational roles in human resources, justifying their high scores.
This ranking aids recruiters in prioritizing the most suitable candidates quickly and efficiently.

# **Method 3 :Candidate Scoring using Qwen3 + LoRA + Unsloth**

we leverage **Qwen3 (Qwen1.5-4B-Instruct)**, a powerful open-weight language model, optimized via **LoRA (Low-Rank Adaptation)** using **Unsloth** for efficient inference and fine-tuning.

⚙️ Key Components:

**Qwen3-4B-Instruct:** A high-quality instruction-tuned model ideal for reasoning and ranking tasks.

**LoRA via Unsloth:** Enables lightweight and fast adaptation of the base model without full fine-tuning, making it efficient even on limited hardware.

**Search-Term Filtering:** Profiles that do not contain the specified search keyword (e.g., "aspiring") are automatically assigned a score of 0, ensuring relevance.

In [2]:
!pip uninstall -y transformers trl unsloth unsloth_zoo

Found existing installation: transformers 4.53.2
Uninstalling transformers-4.53.2:
  Successfully uninstalled transformers-4.53.2
Found existing installation: trl 0.7.11
Uninstalling trl-0.7.11:
  Successfully uninstalled trl-0.7.11
Found existing installation: unsloth 2025.7.3
Uninstalling unsloth-2025.7.3:
  Successfully uninstalled unsloth-2025.7.3
Found existing installation: unsloth_zoo 2025.7.4
Uninstalling unsloth_zoo-2025.7.4:
  Successfully uninstalled unsloth_zoo-2025.7.4


In [3]:
!pip install transformers==4.36.2
!pip install trl==0.7.1
!pip install unsloth unsloth_zoo


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 93.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.2 which is incompatible.


  Using cached unsloth-2025.7.3-py3-none-any.whl.metadata (47 kB)
  Using cached unsloth_zoo-2025.7.4-py3-none-any.whl.metadata (8.1 kB)
  Using cached transformers-4.53.2-py3-none-any.whl.metadata (40 kB)
  Using cached trl-0.19.1-py3-none-any.whl.metadata (10 kB)
  Using cached tokenizers-0.21.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached unsloth-2025.7.3-py3-none-any.whl (297 kB)
Using cached unsloth_zoo-2025.7.4-py3-none-any.whl (162 kB)
Using cached transformers-4.53.2-py3-none-any.whl (10.8 MB)
Using cached tokenizers-0.21.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
Using cached trl-0.19.1-py3-none-any.whl (376 kB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.2
    Uninstalling transformers-4.36.2:
    

1. **transformers==4.36.2:** Provides the core framework for working with large language models (LLMs) from Hugging Face.
2. **trl==0.7.1:** Supports reinforcement learning with human feedback (RLHF) and fine-tuning transformer models using LoRA and PEFT techniques.
3. **unsloth and unsloth_zoo:** Optimize the loading and fine-tuning of LLMs with 4-bit quantization, memory efficiency, and built-in support for LoRA on models like Mistral, Gemma, Qwen, and GROK.

#### load Qwen + LoRA using Unsloth

In [4]:
# ✅ Load Qwen 4B in 4bit with LoRA via Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-unsloth-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)


==((====))==  Unsloth 2025.7.3: Fast Qwen3 patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.7.3 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [15]:

def score_candidate(candidate_profile: str, target_role: str, search_term: str) -> str:
    if search_term.lower() not in candidate_profile.lower():
        return "0"

    prompt = f"""
You are an expert HR recruiter.

Evaluate the following candidate for the role of "{target_role}".
Give a score from 1 to 10 ONLY (just the number, no explanation).

Candidate Profile:
{candidate_profile}

Score:""".strip()

    messages = [{"role": "user", "content": prompt}]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        return_tensors="pt",
        return_dict=True
    )
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    outputs = model.generate(**inputs, max_new_tokens=10, do_sample=False)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    last_line = decoded.strip().split("\n")[-1]
    score = ''.join(filter(str.isdigit, last_line))

    return score if score else "0"



This function uses a large language model to evaluate how well a candidate fits a specific job role, but with an important pre-filtering step:

✅ Function Highlights:
- **Search Filter:**
Candidates are scored only if their profile contains the specified search_term (e.g., "aspiring", "manager"). This helps focus evaluation on relevant profiles, reducing computation and noise.
- **Prompt-Based Evaluation:**
The function constructs a natural language prompt instructing the model to rate the candidate from 1 to 10, purely based on alignment with the target_role.
- **Model Inference:**
1. Uses the Unsloth-optimized tokenizer and model.
2. Applies a structured chat template for better prompt formatting.
3. Extracts the score from the model’s output using string parsing.

In [16]:
search_term = "aspiring"
target_role = "Aspiring Hiring Manager"

df["qwen_score"] = df["candidate_profile"].apply(lambda x: score_candidate(x, target_role, search_term))


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p'

In [17]:
df["qwen_score"] = df["qwen_score"].astype(int)
top_20 = df.sort_values("qwen_score", ascending=False).head(20)
print(top_20[["job_title", "qwen_score"]])


                                            job_title  qwen_score
0   2019 C.T. Bauer College of Business Graduate (...           7
2               Aspiring Human Resources Professional           7
26  Aspiring Human Resources Management student se...           7
6   Student at Humber College and Aspiring Human R...           7
5                 Aspiring Human Resources Specialist           7
71  Business Management Major and Aspiring Human R...           7
65  Experienced Retail Manager and aspiring Human ...           7
99  Aspiring Human Resources Manager | Graduating ...           7
96              Aspiring Human Resources Professional           7
81  Aspiring Human Resources Professional | An ene...           7
78  Liberal Arts Major. Aspiring Human Resources A...           7
75  Aspiring Human Resources Professional | Passio...           7
72  Aspiring Human Resources Manager, seeking inte...           7
12  Human Resources Coordinator at InterContinenta...           0
11  SVP, C

This output represents the **top 20** ranked candidates based on their **Qwen** model-generated scores, filtered and sorted using candidate ranking pipeline. Here's how to interpret it:

- **Score of 7:**
Candidates ranked with a score of **7** have strong alignment with the target role. Most of these profiles explicitly include keywords like **“Aspiring,” “Human Resources,”** or **“Manager”**, showing clear relevance.
- **Score of 0:**
Candidates receiving a score of **0** either:
Do not include the required search term (like “aspiring”), or
Their profile is not semantically aligned with the job role (e.g., they may work in unrelated domains like education, tech, or business analytics).

- Candidates like:

**"Aspiring Human Resources Professional"**

**"Aspiring Human Resources Manager, seeking internship"**

**"Experienced Retail Manager and aspiring HR professional"**

Are scoring **7**, indicating strong alignment with the role **"Aspiring Human Resources Manager"**.

- Candidates like:

**"SVP, CHRO, Marketing & Communications"**

**"Student at Chapman University"**

Score **0**, likely due to lack of role-specific relevance or missing keywords.